<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/vector_stores/CognitiveSearchIndexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# Azure AI Search


## 基本示例

在这个笔记本中，我们将使用一个Paul Graham的文章，将其分成片段，使用Azure OpenAI嵌入模型进行嵌入，将其加载到Azure AI搜索索引中，然后进行查询。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama-index
!pip install wget
%pip install llama-index-vector-stores-azureaisearch
%pip install azure-search-documents==11.4.0
%llama-index-embeddings-azure-openai
%llama-index-llms-azure-openai

In [ ]:
import logging
import sys
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.core import (
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.settings import Settings

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore
from llama_index.vector_stores.azureaisearch import (
    IndexManagement,
    MetadataIndexFieldType,
)

## 设置Azure OpenAI


In [ ]:
aoai_api_key = "YOUR_AZURE_OPENAI_API_KEY"
aoai_endpoint = "YOUR_AZURE_OPENAI_ENDPOINT"
aoai_api_version = "2023-05-15"

llm = AzureOpenAI(
    model="YOUR_AZURE_OPENAI_COMPLETION_MODEL_NAME",
    deployment_name="YOUR_AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME",
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)

# 您需要部署自己的嵌入模型以及自己的聊天完成模型
embed_model = AzureOpenAIEmbedding(
    model="YOUR_AZURE_OPENAI_EMBEDDING_MODEL_NAME",
    deployment_name="YOUR_AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME",
    api_key=aoai_api_key,
    azure_endpoint=aoai_endpoint,
    api_version=aoai_api_version,
)

## 设置Azure AI Search


In [ ]:
search_service_api_key = "YOUR-AZURE-SEARCH-SERVICE-ADMIN-KEY"
search_service_endpoint = "YOUR-AZURE-SEARCH-SERVICE-ENDPOINT"
search_service_api_version = "2023-11-01"
credential = AzureKeyCredential(search_service_api_key)


# 要使用的索引名称
index_name = "llamaindex-vector-demo"

# 使用索引客户端来演示创建索引
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# 使用搜索客户端来演示使用现有索引
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

## 创建索引（如果不存在）


演示了如果不存在的话创建一个名为"llamaindex-vector-demo"的向量索引。该索引具有以下字段：
| 字段名   | OData类型                 |  
|----------|--------------------------|  
| id       | `Edm.String`             |  
| chunk    | `Edm.String`             |  
| embedding| `Collection(Edm.Single)` |  
| metadata | `Edm.String`             |  
| doc_id   | `Edm.String`             |  
| author   | `Edm.String`             |  
| theme    | `Edm.String`             |  
| director | `Edm.String`             |


In [ ]:
metadata_fields = {
    "author": "author",
    "theme": ("topic", MetadataIndexFieldType.STRING),
    "director": "director",
}

vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    filterable_metadata_field_keys=metadata_fields,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1536,
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
)

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

### 加载文档
使用SimpleDirectoryReader加载存储在`data/paul_graham/`中的文档。


In [ ]:
# 加载文档
documents = SimpleDirectoryReader("../data/paul_graham/").load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Settings.llm = llm
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

In [ ]:
# 查询数据
query_engine = index.as_query_engine(similarity_top_k=3)
response = query_engine.query("作者在成长过程中做了什么？")
display(Markdown(f"<b>{response}</b>"))

<b>The author engaged in writing and programming activities during their formative years. They initially wrote short stories and later transitioned to programming on the IBM 1401 using an early version of Fortran. Subsequently, with the advent of microcomputers, the author began programming on a TRS-80, writing simple games, a rocket flight prediction program, and a word processor.</b>

In [ ]:
response = query_engine.query(
    "What did the author learn?",
)
display(Markdown(f"<b>{response}</b>"))

<b>The author learned that the study of philosophy in college did not live up to their expectations, as they found the courses to be boring and lacking in ultimate truths. This led them to switch their focus to AI, which was influenced by a novel featuring an intelligent computer and a PBS documentary showcasing advanced technology.</b>

在 Pandas 中，我们可以使用已经存在的索引来创建新的数据框。这可以通过 `reindex` 方法来实现。让我们来看一个例子：


In [ ]:
index_name = "llamaindex-vector-demo"

metadata_fields = {
    "author": "author",
    "theme": ("topic", MetadataIndexFieldType.STRING),
    "director": "director",
}
vector_store = AzureAISearchVectorStore(
    search_or_index_client=search_client,
    filterable_metadata_field_keys=metadata_fields,
    index_management=IndexManagement.VALIDATE_INDEX,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1536,
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
)

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    [],
    storage_context=storage_context,
)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What was a hard moment for the author?")
display(Markdown(f"<b>{response}</b>"))

<b>The author faced a challenging moment when he couldn't figure out what to do with the early computer he had access to in 9th grade. This was due to the limited options for input and the lack of knowledge in math to do anything interesting with the available resources.</b>

In [ ]:
response = query_engine.query("Who is the author?")
display(Markdown(f"<b>{response}</b>"))

<b>Paul Graham</b>

In [ ]:
import time

query_engine = index.as_query_engine(streaming=True)
response = query_engine.query("What happened at interleaf?")

start_time = time.time()

token_count = 0
for token in response.response_gen:
    print(token, end="")
    token_count += 1

time_elapsed = time.time() - start_time
tokens_per_second = token_count / time_elapsed

print(f"\n\nStreamed output at {tokens_per_second} tokens/s")

The author worked at Interleaf, where they learned several lessons, including the importance of product-focused leadership in technology companies, the drawbacks of code being edited by too many people, the limitations of conventional office hours for optimal hacking, and the risks associated with bureaucratic customers. Additionally, the author discovered the concept that the low end tends to dominate the high end, and that being the "entry level" option can be advantageous.

Streamed output at 99.40073103089465 tokens/s


要将文档添加到现有的索引中，可以使用以下代码示例：

```python
from elasticsearch import Elasticsearch

# 连接到Elasticsearch实例
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# 索引名称
index_name = "your_index_name"

# 文档数据
document = {
    "title": "Sample document",
    "content": "This is a sample document for indexing."
}

# 将文档添加到索引
es.index(index=index_name, body=document)
```

在这个示例中，我们首先连接到Elasticsearch实例，然后指定要添加文档的索引名称和文档数据。最后，我们使用`es.index`方法将文档添加到指定的索引中。


In [ ]:
response = query_engine.query("What colour is the sky?")
display(Markdown(f"<b>{response}</b>"))

<b>Blue</b>

In [ ]:
from llama_index.core import Document

index.insert_nodes([Document(text="The sky is indigo today")])

In [ ]:
response = query_engine.query("What colour is the sky?")
display(Markdown(f"<b>{response}</b>"))

<b>The sky is indigo today.</b>

## 过滤


In [ ]:
from llama_index.core.schema import TextNode

nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
        },
    ),
    TextNode(
        text="Inception",
        metadata={
            "director": "Christopher Nolan",
        },
    ),
]

In [ ]:
index.insert_nodes(nodes)

In [ ]:
from llama_index.core.vector_stores.types import (
    MetadataFilters,
    ExactMatchFilter,
)


filters = MetadataFilters(
    filters=[ExactMatchFilter(key="theme", value="Mafia")]
)

retriever = index.as_retriever(filters=filters)
retriever.retrieve("What is inception about?")

[NodeWithScore(node=TextNode(id_='049f00de-13be-4af3-ab56-8c16352fe799', embedding=None, metadata={'director': 'Francis Ford Coppola', 'theme': 'Mafia'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='ad2a08d4364262546db9711b915348d43e0ccc41bd8c3c41775e133624e1fa1b', text='The Godfather', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8120511)]

## 查询模式
支持四种查询模式：DEFAULT（向量搜索）、SPARSE（稀疏）、HYBRID（混合）和SEMANTIC_HYBRID（语义混合）。


### 执行向量搜索


In [ ]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode

# 默认的检索器
default_retriever = index.as_retriever(
    vector_store_query_mode=VectorStoreQueryMode.DEFAULT
)
response = default_retriever.retrieve("Inception是关于什么的？")

# 遍历响应中的每个NodeWithScore
for node_with_score in response:
    node = node_with_score.node  # TextNode对象
    score = node_with_score.score  # 相似度分数
    chunk_id = node.id_  # 块ID

    # 从节点中提取相关的元数据
    file_name = node.metadata.get("file_name", "未知")
    file_path = node.metadata.get("file_path", "未知")

    # 从节点中提取文本内容
    text_content = node.text if node.text else "无可用内容"

    # 以用户友好的格式打印结果
    print(f"分数: {score}")
    print(f"文件名: {file_name}")
    print(f"ID: {chunk_id}")
    print("\n提取的内容:")
    print(text_content)
    print("\n" + "=" * 40 + " 结果结束 " + "=" * 40 + "\n")

Score: 0.8748552
File Name: Unknown
Id: bae0df75-ff37-4725-b659-b9fd8bf2ef3c

Extracted Content:
Inception

======================================== End of Result ========================================

Score: 0.8155207
File Name: paul_graham_essay.txt
Id: ae5aee85-a083-4141-bf75-bbb872f53760

Extracted Content:
It's not that unprestigious types of work are good per se. But when you find yourself drawn to some kind of work despite its current lack of prestige, it's a sign both that there's something real to be discovered there, and that you have the right kind of motives. Impure motives are a big danger for the ambitious. If anything is going to lead you astray, it will be the desire to impress people. So while working on things that aren't prestigious doesn't guarantee you're on the right track, it at least guarantees you're not on the most common type of wrong one.

Over the next several years I wrote lots of essays about all kinds of different topics. O'Reilly reprinted a collecti

### 执行混合搜索


In [ ]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode

hybrid_retriever = index.as_retriever(
    vector_store_query_mode=VectorStoreQueryMode.HYBRID
)
hybrid_retriever.retrieve("What is inception about?")

[NodeWithScore(node=TextNode(id_='bae0df75-ff37-4725-b659-b9fd8bf2ef3c', embedding=None, metadata={'director': 'Christopher Nolan'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='9792a1fd7d2e1a08f1b1d70a597357bb6b68d69ed5685117eaa37ac9e9a3565e', text='Inception', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.03181818127632141),
 NodeWithScore(node=TextNode(id_='ae5aee85-a083-4141-bf75-bbb872f53760', embedding=None, metadata={'file_path': '..\\data\\paul_graham\\paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75395, 'creation_date': '2023-12-12', 'last_modified_date': '2023-12-12', 'last_accessed_date': '2024-02-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name'

### 使用语义重新排序执行混合搜索
这种模式将语义重新排序应用于混合搜索结果，以提高搜索相关性。

请查看此链接以获取更多详细信息：https://learn.microsoft.com/azure/search/semantic-search-overview


In [ ]:
hybrid_retriever = index.as_retriever(
    vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID
)
hybrid_retriever.retrieve("What is inception about?")

[NodeWithScore(node=TextNode(id_='bae0df75-ff37-4725-b659-b9fd8bf2ef3c', embedding=None, metadata={'director': 'Christopher Nolan'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='9792a1fd7d2e1a08f1b1d70a597357bb6b68d69ed5685117eaa37ac9e9a3565e', text='Inception', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=2.3949906826019287),
 NodeWithScore(node=TextNode(id_='fc9782a2-c255-4265-a618-3a864abe598d', embedding=None, metadata={'file_path': '..\\data\\paul_graham\\paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75395, 'creation_date': '2023-12-12', 'last_modified_date': '2023-12-12', 'last_accessed_date': '2024-02-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name',